In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0, "/home/huseyinalecakir/Security/SentenceOnlyAttentionDynet")


In [ ]:
class Arguments:
    permission_type = "READ_CONTACTS"
    saved_data = "/home/huseyinalecakir/datasets/saved-parameters/saved-data/ac-net/embeddings-sentences-w2i.pickle"
    model_checkpoint = "/home/huseyinalecakir/datasets/saved-parameters/saved-models/SentenceOnlyAttentionDyNET-bidirectional-gru-READ_CONTACTS.pt"
    outdir = "/home/huseyinalecakir/Security/output/SentenceOnlyAttentionDyNET/bidirectional-gru/READ_CONTACTS.out"
    encoder_dir = "bidirectional"
    encoder_type = "gru"
    num_epoch = 1
    stemmer = "porter"
    embedding_size = 300
    hidden_size = 128
    attention_size = 128
    init_weight = 0.08
    output_size = 1
    dropout = 0
    print_every = 1000




In [1]:
model = load_model(args)

NameError: name 'load_model' is not defined

In [ ]:
args = Arguments()


In [ ]:
p_sent = NLPUtils.preprocess_sentence("read your contacts", args.stemmer)

In [ ]:
predict_raw_sentence(model, p_sent)

In [ ]:
import nltk

In [ ]:
import benepar
parser = benepar.Parser("benepar_en2")



In [ ]:
def extract_nodes(parent):
    parts = {}
    parts[parent.label()] = " ".join(parent.leaves())
    for node in parent:
        if type(node) is nltk.Tree:
            parts[node.label()] = " ".join(node.leaves())
    return parts

In [ ]:
sys.path.insert(0, "/home/huseyinalecakir/NLG/codes/scripts/google_apis_crawler")


In [ ]:
from google_apis_crawler.models import Class, Method, db_connect, create_table, Description
from sqlalchemy.orm import sessionmaker


In [ ]:
engine = db_connect()
create_table(engine)
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
methods = session.query(Method).all()

In [ ]:
for method in methods:
    description = method.description
    try:
        tree = parser.parse(description)
    except ValueError:

    nodes = extract_nodes(tree)
    s = nodes.get("S", "-")
    np = nodes.get("NP", "-")
    vp = nodes.get("VP", "-")
    s_p, np_p, vp_p = -1, -1, -1

    if s != "-":
        sent = NLPUtils.preprocess_sentence(s, args.stemmer)
        s_p = predict_raw_sentence(model,sent)
    if np != "-":
        sent = NLPUtils.preprocess_sentence(np, args.stemmer)
        np_p = predict_raw_sentence(model,sent)
    if vp != "-":
        sent = NLPUtils.preprocess_sentence(vp, args.stemmer)
        vp_p = predict_raw_sentence(model,sent)

    description = Description(s, s_p, np, np_p, vp, vp_p)
    print(description)
    method.descriptions.append(description)
    try:
        session.add(description)
        session.commit()

    except:
        print("Database Error.")
        session.rollback()
        raise

    finally:
        print("OUT")
